In [254]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [244]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import json
import ast

API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.sportradar.com/tennis/trial/v3/en/"
ENDPOINT = "/competitions.json"
   
def fetch_competitions():
    url = f"{BASE_URL}{ENDPOINT}?api_key={API_KEY}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def process_competitions(data):
    competitions = data.get("competitions", [])
    df = pd.json_normalize(competitions)
    return df

competitions_data = fetch_competitions()
competitions_df = process_competitions(competitions_data)
competitions_df.to_csv("competitions.csv", index=False)
print("Competitions data saved!")

Competitions data saved!


In [245]:
ENDPOINT = "/complexes.json"

def fetch_complexes():
    url = f"{BASE_URL}{ENDPOINT}?api_key={API_KEY}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def process_complexes(data):
    complexes = data.get("complexes", [])
    df = pd.json_normalize(complexes)
    return df

complexes_data = fetch_complexes()
complexes_df = process_complexes(complexes_data)
complexes_df.to_csv("complexes.csv", index=False)
print("Complexes data saved!")

Complexes data saved!


In [246]:
ENDPOINT = "/double_competitors_rankings.json"

def fetch_rankings():
    url = f"{BASE_URL}{ENDPOINT}?api_key={API_KEY}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def process_rankings(data):
    rankings = data.get("rankings", [])
    df = pd.json_normalize(rankings)
    return df

rankings_data = fetch_rankings()
rankings_df = process_rankings(rankings_data)
rankings_df.to_csv("rankings.csv", index=False)
print("Rankings data saved!")

Rankings data saved!


In [247]:
username = os.getenv("user_name")
password = os.getenv("password")
host = os.getenv("host")
database = os.getenv("database")

engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}")

try:
    with engine.connect() as connection:
        print("Database connected successfully!")
except Exception as e:
    print(f"Error: {e}")

Database connected successfully!


In [248]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()

categories = Table(
    'categories', metadata,
    Column('id', String(50), primary_key=True),
    Column('name', String(100), nullable=False),
)

competitions = Table(
    'competitions', metadata,
    Column('id', String(50), primary_key=True),
    Column('name', String(100), nullable=False),
    Column('type', String(20), nullable=False),
    Column('gender', String(10), nullable=False),
    Column('category.id', String(50), ForeignKey('categories.id')),
    Column('category.name', String(100), nullable=False),
    Column('level', String(50), nullable=True),
    Column('parent_id', String(50), nullable=True),
)

complexes = Table(
    'complexes', metadata,
    Column('id', String(50), primary_key=True),
    Column('name', String(100), nullable=False),
    Column('venues', String(500), nullable=True),
)

venues = Table(
    'venues', metadata,
    Column('id', String(50), primary_key=True),
    Column('name', String(100), nullable=False),
    Column('city_name', String(100), nullable=False),
    Column('country_name', String(100), nullable=False),
    Column('country_code', String(3), nullable=False),
    Column('timezone', String(100), nullable=False),
    Column('complex_id', String(50), ForeignKey('complexes.id')),
)

competitors = Table(
    'competitors_table', metadata,
    Column('id', String(50), primary_key=True),
    Column('name', String(100), nullable=False),
    Column('country', String(255), nullable=False),
    Column('country_code', String(255), nullable=False),
    Column('abbreviation', String(255), nullable=False),
)

rankings = Table(
    'competitor_rankings', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('rank', Integer, nullable=False),
    Column('movement', Integer, nullable=False),
    Column('points', Integer, nullable=False),
    Column('competitions_played', Integer, nullable=False),
    Column('competitor_id', String(50), ForeignKey('competitors_table.id')),
)

metadata.drop_all(engine)
metadata.create_all(engine)

print("Tables created successfully!")

Tables created successfully!


In [249]:
categories_df = competitions_df[['category.id', 'category.name']].drop_duplicates()
categories_df.rename(columns={'category.id': 'id', 'category.name': 'name'}, inplace=True)

In [250]:
complexes_data = pd.read_csv("complexes.csv")

venues_list = []
for _, row in complexes_data.iterrows():
    if not pd.isna(row['venues']):
        venues = ast.literal_eval(row['venues'])
        for venue in venues:
            venue['complex_id'] = row['id']
            venues_list.append(venue)

venues_df = pd.DataFrame(venues_list)
venues_df = venues_df.drop(columns=["capacity", "map_coordinates"])

In [251]:
complexes_df = complexes_df.drop(columns=["venues"])
complexes_df

,id,name
0,sr:complex:705,Nacional
1,sr:complex:1078,Estadio de la Cartuja
2,sr:complex:1495,Sibur Arena
3,sr:complex:2375,Complexo de Tenis do Jamor
4,sr:complex:4032,Shree Shiv Chhatrapati Sports Complex
...,...,...
610,sr:complex:81543,Headington Family Tennis Center
611,sr:complex:81613,Royal SC Tennis
612,sr:complex:81635,LSU
613,sr:complex:81651,Robson Tennis Center


In [252]:
rankings_data = pd.read_csv("rankings.csv")
competitor_rankings = []
table = []

for _, row in rankings_data.iterrows():
    rankings = ast.literal_eval(row['competitor_rankings'])
    for ranking in rankings:
        competitor = ranking['competitor']
        competitor_rankings.append({
            'rank': ranking['rank'],
            'movement': ranking['movement'],
            'points': ranking['points'],
            'competitions_played': ranking['competitions_played'],
            'competitor_id': competitor['id']
        })
        table.append({
            'id': competitor['id'],
            'name': competitor['name'],
            'country': competitor['country'],
            'country_code': competitor.get('country_code', 'N/A'),
            'abbreviation': competitor['abbreviation']
        })
        # competitors_df = pd.DataFrame([competitor])

competitors_df = pd.DataFrame(table)
rankings_df = pd.DataFrame(competitor_rankings)
print("Rankings data inserted successfully!")
print(competitions_df.head())
print(rankings_df.head())

Rankings data inserted successfully!
                    id                               name     type gender  \
0   sr:competition:620                         Hopman Cup    mixed  mixed   
1   sr:competition:660                     World Team Cup    mixed    men   
2   sr:competition:990         ATP Challenger Tour Finals  singles    men   
3  sr:competition:1207  Championship International Series  singles  women   
4  sr:competition:2100                          Davis Cup    mixed    men   

       category.id category.name              level            parent_id  
0  sr:category:181    Hopman Cup                NaN                  NaN  
1    sr:category:3           ATP            atp_250                  NaN  
2   sr:category:72    Challenger                NaN  sr:competition:6239  
3    sr:category:6           WTA  wta_championships                  NaN  
4   sr:category:76     Davis Cup                NaN                  NaN  
   rank  movement  points  competitions_played    

In [253]:
categories_df.to_sql('categories', con=engine, if_exists='append', index=False)
competitions_df.to_sql('competitions', con=engine, if_exists='append', index=False)

complexes_df.to_sql('complexes', con=engine, if_exists='append', index=False)
venues_df.to_sql('venues', con=engine, if_exists='append', index=False)

competitors_df.to_sql('competitors_table', con=engine, if_exists='append', index=False)
rankings_df.to_sql('competitor_rankings', con=engine, if_exists='append', index=False)

print("Data inserted into tables successfully!")

Data inserted into tables successfully!
